In [10]:
# pip install biopython

In [136]:
import datetime
from Bio.Seq import Seq, back_transcribe

# 1. Class that describes hamsters.

Python class for hamsters.\
For creation "hamster" object it is nedeed to know its name and age (in months).\
Optionally, one can specify the type of hamster (for example, Syrian or Chinese). Default "Not defined".

In [137]:

class Hamster:
    '''
    Python class for hamsters.
    '''

    def __init__(self, name, age, hamster_type='Not defined'):
        '''
        For creation "hamster" object it is nedeed to know its name and age (in months).
        Optionally, one can specify the type of hamster (for example, Syrian or Chinese). Default "Not defined".
        '''
        self.name = name
        self.hamster_type = hamster_type
        self.age = age
        
        if self.age <= 4:
            self.stage = 'young'            
            
        elif self.age >= 12:
            self.stage = 'old'
        
        else:
            self.stage = 'adult'
        

    def life_stage(self):
        '''
        Method returns the age of a hamster: young, adult or old.
        '''        
        return f"{self.name} is an {self.stage} hamster. It's {self.age} months."


    def is_active(self):
        '''
        Checks if the hamster is probably asleep now. The system time is used. Return the string with state.
        '''
        time_now = datetime.datetime.now().hour
        
        if time_now > 7 and time_now < 20:
            return f'{self.name} is probably sleeping now.'
            
        else:
            return f'{self.name} is probably active now and wants treats.'


    def properties(self):
        '''
        Return string with the basic parameters of the hamster.
        '''
        if self.hamster_type == 'Not defined':            
            return f"The hamster\'s name is {self.name}. It\'s {self.stage} hamster, {self.age} months."
        else:
            return f"The hamster\'s name is {self.name}. It\'s {self.stage} {self.hamster_type} hamster, {self.age} months."

In [138]:
Bun = Hamster('Bun', 18, 'Djungarian hamster')
print(Bun.is_active())
print(Bun.properties())
print(Bun.life_stage())

Bun is probably active now and wants treats.
The hamster's name is Bun. It's old Djungarian hamster hamster, 18 months.
Bun is an old hamster. It's 18 months.


# 2. Class that describes RNA sequence data

Последовательности РНК - строки, поэтому класс наследован от str.

In [154]:
class RNASequence():
    '''
    Class for operations with RNA sequences. Parent class - str.
    '''
    
    def __init__(self, rna_sequence):
        self.rna_sequence = rna_sequence
        
    def RNA_translation(self):
        '''
        Translation - returns a string corresponding to the protein from the RNA,
        according to the standard code, using Biopython.
        '''
        self.protein = Bio.Seq.translate(self.rna_sequence)
        return self.protein
        
    def RNA_to_DNA(self):
        '''
        Reverse transcription method - returns a string corresponding to the DNA from RNA.
        '''
        self.template_dna = Bio.Seq.back_transcribe(self.rna_sequence)
        return self.template_dna

In [158]:
rna_seq = RNASequence('AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG')
print(rna_seq.RNA_translation())
print(rna_seq.RNA_to_DNA())


rna_seq_2 = RNASequence('AUGC')
print(rna_seq_2.RNA_translation())
print(rna_seq_2.RNA_to_DNA())

MAIVMGR*KGAR*
ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
M
ATGC
